<a href="https://colab.research.google.com/github/micsiler/Covid-19-Fake-News-Project/blob/main/Covid19FakeNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Covid-19 Fake New Detector (B365 Project)
Authors: Michael Siler, Wen Cheng, Bowen (Kevin) Shen, Arjun Varadhan, Julianne Sendon

Data Set Provided By: https://data.mendeley.com/datasets/zwfdmp5syg/1/files/3063167e-1d3b-4604-a630-16016a84e8db

Presentation: https://docs.google.com/presentation/d/1C77iplRyZ_nKjfDDCcN7TIeSMlH6VLIMH-Noy4Cw7K4/edit?usp=sharing

Description: This is a project for B365 (Data Analysis and Mining). This is an analysis of multiple methods of classification on fake covid-19 news datasets. 


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

import os

In [ ]:
import io

from google.colab import files
uploaded = files.upload()
df = pd.read_excel(io.BytesIO(uploaded['fake_new_dataset.xlsx']))

df

Saving fake_new_dataset.xlsx to fake_new_dataset.xlsx


,Unnamed: 0,title,text,subcategory,label
0,0,FACEBOOK DELETES MICHIGAN ANTI-LOCKDOWN GROUP ...,Facebook has shuttered a popular group for Mic...,false news,0
1,1,Other Viewpoints: COVID-19 is worse than the flu,We can now officially put to rest all comparis...,true,1
2,2,Bermuda's COVID-19 cases surpass 100,The Ministry of Health in Bermuda has confirme...,true,1
3,3,Purdue University says students face 'close to...,"Purdue University President Mitch Daniels, the...",partially false,0
4,4,THE HIGH COST OF LOCKING DOWN AMERICA: “WE’VE ...,Locking down much of the country may have help...,false news,0
...,...,...,...,...,...
3114,2998,2019-nCoV: Health dept. on full alert,Quarantine advised for 80 persons across the S...,true,1
3115,2999,Screening machine for corona virus launched at...,South Sudan has begun screening all travelers ...,true,1
3116,3000,Coronavirus (2019-nCoV),Coronavirus (2019-nCoV) is a new respiratory i...,true,1
3117,3001,Characterizing Patients Hospitalized With COVI...,Characterizing Patients Hospitalized With COVI...,false news,0


In [ ]:
df.isna().sum() ## there is no need to remove null value for our dataset.

Unnamed: 0     0
title          1
text           0
subcategory    0
label          0
dtype: int64

In [ ]:
print(sum(df["label"]==1))
print(sum(df["label"]==0)) ##imbalabced, and we decide to use all 0 label and half 1 label

2061
1058


In [ ]:
labels=df.label
labels.head()  ## if the the news is real, the label is 1; if the news is fake or partially fake ,the lebel is 0

0    0
1    1
2    1
3    0
4    0
Name: label, dtype: int64

In [ ]:
np.random.seed(500)


In [ ]:
df[df['label']==0]

,Unnamed: 0,title,text,subcategory,label
0,0,FACEBOOK DELETES MICHIGAN ANTI-LOCKDOWN GROUP ...,Facebook has shuttered a popular group for Mic...,false news,0
3,3,Purdue University says students face 'close to...,"Purdue University President Mitch Daniels, the...",partially false,0
4,4,THE HIGH COST OF LOCKING DOWN AMERICA: “WE’VE ...,Locking down much of the country may have help...,false news,0
5,5,NY TIMES WORRIES HALF OF AMERICANS WILL REFUSE...,The New York Times has published an article by...,false news,0
7,7,CDC adds 6 new possible symptoms of Covid-19,"When the coronavirus outbreak began, the Cente...",false news,0
...,...,...,...,...,...
3103,2987,Twinbrook ‘pausing’ COVID-19 plan after criticism,"Apr 24, 2020 at 4:31 PM Nursing home faced bac...",false news,0
3109,2993,"Experts suggest ventilation, not isolation is ...",The Coronavirus epidemic for the first time re...,false news,0
3110,2994,Corona virus and pentagon Quarantine help? Com...,Here is the technique we use to heal unseen ne...,false news,0
3117,3001,Characterizing Patients Hospitalized With COVI...,Characterizing Patients Hospitalized With COVI...,false news,0


In [ ]:
## solve the problem of imbalance
df_new = pd.DataFrame()
df_new['title']=""
df_new['text']=""
df_new['subcategory']=""
df_new['label']=0

label1_new=df[df['label']==1].sample(n=1060,random_state=2)
label1_new = label1_new.iloc[:,1:]

label0_new=df[df['label']==0]
label0_new = label0_new.iloc[:,1:]

df_new = df_new.append(label1_new)
df_new = df_new.append(label0_new)

## this is the new dataset which is not imbalanced

df_new.reset_index(drop=True, inplace=True)
df_new

,title,text,subcategory,label
0,Formula 1 Consider Changing the Chinese GP Sch...,Home Sports Formula 1 Consider Changing the Ch...,true,1
1,Belarus' Healthcare Ministry: Over 120 people ...,The National Research Center for Epidemiology ...,true,1
2,Mumtaz Ali calls for raising awareness about N...,"Health Ministry Focal person , Dr Mumtaz Ali K...",true,1
3,The Definitive Guide to corona virus ped,A coronavirus is actually a kind of common vir...,true,1
4,In the middle of the corona epidemic: Nurses w...,A nurse in Wuhan forced to marry fiancé via vi...,true,1
...,...,...,...,...
2113,Twinbrook ‘pausing’ COVID-19 plan after criticism,"Apr 24, 2020 at 4:31 PM Nursing home faced bac...",false news,0
2114,"Experts suggest ventilation, not isolation is ...",The Coronavirus epidemic for the first time re...,false news,0
2115,Corona virus and pentagon Quarantine help? Com...,Here is the technique we use to heal unseen ne...,false news,0
2116,Characterizing Patients Hospitalized With COVI...,Characterizing Patients Hospitalized With COVI...,false news,0


In [ ]:
# split the train and test dataset
x_train,x_test,y_train,y_test=train_test_split(df_new['text'], df_new['label'], test_size=0.3, random_state=17)

In [ ]:
## tf-idf vectorize , tokenizer??

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)

tfidf_train=tfidf_vectorizer.fit_transform(x_train) 
tfidf_test=tfidf_vectorizer.transform(x_test)


#Passive Agressive Classifier

In [ ]:
## initialize passive aggressive classifier
from sklearn.linear_model import PassiveAggressiveClassifier

pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train,y_train)

y_pred_pac=pac.predict(tfidf_test)

# confusion matrix
from sklearn.metrics import accuracy_score, confusion_matrix

print(confusion_matrix(np.array(y_test),y_pred_pac, labels=[0,1]))

print('accuracy of PAC = ', round((233+239)/636,3))  ## the accuracy is 0.742


[[215 101]
 [ 64 256]]
accuracy of PAC =  0.742


#Logistic Regression

In [ ]:
# Logistics regression
from sklearn.linear_model import LogisticRegression

logit = LogisticRegression(C=5e1, solver='lbfgs', multi_class='multinomial', random_state=17, n_jobs=4)
logit.fit(tfidf_train, y_train)

y_pred_LR = logit.predict(tfidf_test)

#confusion matrix

print(confusion_matrix(np.array(y_test),y_pred_LR, labels=[0,1]))

print("Accuracy of LR = ", round((240+250)/636,3))
# The accuracy of LR is 0.77

[[240  76]
 [ 70 250]]
Accuracy of LR =  0.77


#SVM (Support Vector Machine)

In [ ]:
# SVM - support vector machine
from sklearn import model_selection, naive_bayes, svm
from sklearn.svm import SVC

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(tfidf_train, y_train)

y_pred_SVM = SVM.predict(tfidf_test)

#confusion matrix
print(confusion_matrix(np.array(y_test),y_pred_SVM, labels=[0,1]))

print("Accuracy of SVM = ", round((236+261)/636,3))
# The accuracy of SVM is 0.781

print("SVM Accuracy Score -> ",accuracy_score(y_pred_SVM, y_test))


[[236  80]
 [ 59 261]]
Accuracy of SVM =  0.781
SVM Accuracy Score ->  0.7814465408805031


#Naive Baye's

In [ ]:
# Naive Baye 
Naive = naive_bayes.MultinomialNB()
Naive.fit(tfidf_train, y_train)

y_pred_NB = Naive.predict(tfidf_test)

#confusion matrix
print(confusion_matrix(np.array(y_test),y_pred_NB, labels=[0,1]))

print("Accuracy of NB = ", round((194+286)/636,3))
# The accuracy of NB is 0.755

print("Naive Bayes Accuracy Score -> ",accuracy_score(y_pred_NB, y_test))

[[194 122]
 [ 34 286]]
Accuracy of NB =  0.755
Naive Bayes Accuracy Score ->  0.7547169811320755


#KNN (K-Nearest Neighbor)

In [ ]:
# KNN
'''
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=3)
knn.fit(tfidf_train)

y_pred_knn = knn.predict(tfidf_test)

#confusion matrix
print(confusion_matrix(np.array(y_test),y_pred_knn, labels=[0,1]))

print("Accuracy of KNN = ", round((20+629)/936,3))
# The accuracy of KNN is 0.

print("KNN Accuracy Score -> ",accuracy_score(y_pred_knn, y_test))
'''

'\nfrom sklearn.neighbors import KNeighborsRegressor\n\nknn = KNeighborsRegressor(n_neighbors=3)\nknn.fit(tfidf_train)\n\ny_pred_knn = knn.predict(tfidf_test)\n\n#confusion matrix\nprint(confusion_matrix(np.array(y_test),y_pred_knn, labels=[0,1]))\n\nprint("Accuracy of KNN = ", round((20+629)/936,3))\n# The accuracy of KNN is 0.\n\nprint("KNN Accuracy Score -> ",accuracy_score(y_pred_knn, y_test))\n'

In [ ]:
## Let's consider why true positive (real =1, pred =1) case for some technique like Naive Baye's Classifier is less.
